In [ ]:

import plotly.express as px
import pandas as pd

import sys; sys.path.append('../..')
from src.results import read_results

trades = read_results.get_trades("ApplesOptions")
# trades = read_results.get_trades("ApplesSellEODOptions")
print(len(list(trades)))

In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
import datetime
import typing
import collections
import itertools
from src.backtest import exiters, execute_exits

tps = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
sls = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
# sls = [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01]
points = []
for sl in sls:
    rois_by_time = {}

    bracketed_trades = []
    for trade in trades:
        # print('    ', trade.get_start())
        exiter = exiters.StopLossTrailingPercentageExiter(sl)
        # t = execute_exits.find_exit_within_trade_timeframe(trade, exiter)
        t = execute_exits.find_exit_within_trade_day(trade, exiter)
        bracketed_trades.append(t)

    profit = sum(t.get_profit_loss() for t in bracketed_trades)
    print(sl, collections.Counter(t.is_win() for t in bracketed_trades), profit)


    for t in bracketed_trades:
        key = t.get_start().time()
        rois_by_time[key] = rois_by_time.get(key, []) + [t.get_profit_loss()]

    points.append({
        'sl': sl,
        'profit': profit,
        "rois_by_time": rois_by_time,
    })


In [ ]:
def group_rois_by_time(rois_by_time):
    grouped_rois = {}
    for time, rois in rois_by_time.items():
        new_time = datetime.time(time.hour, time.minute // 15 * 15)
        grouped_rois[new_time] = grouped_rois.get(new_time, []) + rois
    return grouped_rois

for point in points:
    for time, rois in sorted(group_rois_by_time(point["rois_by_time"]).items()):
        print(time, len(rois), sum(rois) / len(rois))
    
    fig = px.histogram(pd.DataFrame([
        {
            'time': time,
            'freq': len(rois),
            'roi': sum(rois) / len(rois),
        }
        for time, rois in sorted(group_rois_by_time(point["rois_by_time"]).items())
    ]), x="time", y="roi", title=f"SL: {point['sl']:%}")
    fig.show()


In [ ]:
assert False
# CODE not updated yet:

In [ ]:

fig = px.density_heatmap(pd.DataFrame(points), x="sl", y="tp", z="profit", nbinsx=len(sls), nbinsy=len(tps))
fig.show()

In [ ]:
for sl, tp in itertools.product(sls, tps):
    bracket_set= typing.cast(
        list[brackets_historical.Bracket],
        [{'take_profit_percentage': tp, 'stop_loss_percentage': sl, 'until': datetime.time(15, 59)}]
    )
    bracketed_trades = [brackets_historical.get_bracketed_virtual_trade(t, bracket_set) for t in trades]
    fig = px.histogram(
        pd.DataFrame([{'roi': t.get_profit_loss() / t.get_value_spent()} for t in bracketed_trades]),
        x='roi',
        title=f'Bracketed trades sl={sl} tp={tp}',
        marginal='rug',
        range_x=[-1, 1],
    )
    fig.show()
    print("^^^", sl, tp, collections.Counter(t.is_win() for t in bracketed_trades),
          sum(t.get_profit_loss() for t in bracketed_trades))